    For Infosys, Live buy sell signals generated 
    based on 5-20 SMA cross-over

    My respect towrds - krishnavelu@aliceblue - for this program and
    implementing aliceblue library in a very simple manner. 
    
    And special thanks to Aliceblue, for providing this API in free of cost. 

In [1]:
# importing packages           
import logging
import datetime
import statistics
from time import sleep
import account_details    # this contains username, password, api details
from alice_blue import *

In [ ]:
# Config - Username, password, API details loaded from account_details file
username = account_details.usr_name
password = account_details.pwd
app_id = account_details.app_id
api_secret = account_details.api_scrt
twoFA = account_details.tfa
EMA_CROSS_SCRIP = 'INFY'
logging.basicConfig(level=logging.DEBUG)        # Optional for getting debug messages.
# Config

In [ ]:
ltp = 0
socket_opened = False
alice = None
def event_handler_quote_update(message):
    global ltp
    ltp = message['ltp']

In [ ]:
def open_callback():
    global socket_opened
    socket_opened = True

In [ ]:
# Buy order 
def buy_signal(ins_scrip):
    global alice
    alice.place_order(transaction_type = TransactionType.Buy,
                         instrument = ins_scrip,
                         quantity = 1,
                         order_type = OrderType.Market,
                         product_type = ProductType.Intraday,
                         price = 0.0,
                         trigger_price = None,
                         stop_loss = None,
                         square_off = None,
                         trailing_sl = None,
                         is_amo = False)

In [ ]:
# Sell order
def sell_signal(ins_scrip):
    global alice
    alice.place_order(transaction_type = TransactionType.Sell,
                         instrument = ins_scrip,
                         quantity = 1,
                         order_type = OrderType.Market,
                         product_type = ProductType.Intraday,
                         price = 0.0,
                         trigger_price = None,
                         stop_loss = None,
                         square_off = None,
                         trailing_sl = None,
                         is_amo = False)

In [ ]:
def main():
    global socket_opened
    global alice
    global username
    global password
    global twoFA
    global api_secret
    global EMA_CROSS_SCRIP
    
    minute_close = []
    
    # Access token is fetched from aliceblue
    access_token =  AliceBlue.login_and_get_access_token(username=username, password=password, twoFA=twoFA,  
                                                         api_secret=api_secret, app_id = app_id)
    # alice object is created
    alice = AliceBlue(username=username, password=password, access_token=access_token, 
                      master_contracts_to_download=['NSE'])
    
    
    print('User Profile: ', alice.get_profile()) # get profile
    print('User Balance: ', alice.get_balance()) # get balance / margin limits
    # print(alice.get_daywise_positions()) # get daywise positions
    # print(alice.get_netwise_positions()) # get netwise positions
    # print(alice.get_holding_positions()) # get holding positions
    
    ins_scrip = alice.get_instrument_by_symbol('NSE', EMA_CROSS_SCRIP)
    
    # Open the socket
    socket_opened = False
    alice.start_websocket(subscribe_callback=event_handler_quote_update,
                          socket_open_callback=open_callback,
                          run_in_background=True)
    
    while(socket_opened==False):    # wait till socket open & then subscribe
        pass
    alice.subscribe(ins_scrip, LiveFeedType.COMPACT)  # instrument is subscribed
    
    # Each minute data is recorded here.
    # Logic for buy - if SMA5 > SMA20
    # Logic for Sell - if SMA20 > SMA5
    # Sleep() is used program will not letgo in between
    current_signal = ''
    while True:
        if(datetime.datetime.now().second == 0):   # recorded data in each minute
            minute_close.append(ltp)
            print('Minute data: ', ltp)
            if(len(minute_close) > 20):
                sma_5 = statistics.mean(minute_close[-5:])
                sma_20 = statistics.mean(minute_close[-20:])
                if(current_signal != 'buy'):
                    # if(sma_5 > sma_20):
                    #     buy_signal(ins_scrip)
                    current_signal = 'buy'
                    print('Buy')
                if(current_signal != 'sell'):
                    # if(sma_5 < sma_20):
                    #     sell_signal(ins_scrip)
                    current_signal = 'sell'
                    print('Sell')
            sleep(1)
        sleep(0.2)  # sleep for 200ms

In [ ]:
if(__name__ == '__main__'):
    main()